<a href="https://colab.research.google.com/github/KavehKadkhoda/Sentiment-Analysis/blob/main/8_Find_tweets_that_are_similar_to_a_given_tweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# LSH and document search
# We will implement a more efficient version of k-nearest neighbors using locality sensitive hashing. 
# I will then apply this to document search.

# Process the tweets and represent each tweet as a vector (represent a document with a vector embedding).
# Use locality sensitive hashing and k nearest neighbors to find tweets that are similar to a given tweet.

In [2]:
#  positive_tweets.json
#  negative_tweets.json

# These were collected in July 2015 by searching against the following strings:

# positive
# -------
#     ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
#     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
#     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
#     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
#     '<3'
#     }

# negative
# ------
#     ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
#     ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
#     ':c', ':{', '>:\\', ';('
   


In [1]:
#The sample dataset from NLTK is separated into positive and negative tweets. 
#It contains 5000 positive tweets and 5000 negative tweets exactly.
#It is just because balanced datasets simplify the design of most computational methods that are required for sentiment analysis. 
#You can download the dataset in your workspace (or in your local computer) by doing:

import nltk # Python library for NLP

# downloads sample twitter dataset.
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [2]:
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK


# get the positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
all_tweets = all_positive_tweets + all_negative_tweets

In [6]:
#  Getting the document embeddings
# Bag-of-words (BOW) document models
# Text documents are sequences of words.

# The ordering of words makes a difference. For example, sentences "Apple pie is better than pepperoni pizza."
# and "Pepperoni pizza is better than apple pie" have opposite meanings due to the word ordering.
# However, for some applications, ignoring the order of words can allow us to train an efficient and still effective model.
# This approach is called Bag-of-words document model.

# Document embeddings
# Document embedding is created by summing up the embeddings of all words in the document.
# If we don't know the embedding of some word, we can ignore that word.


# The function get_document_embedding() encodes entire document as a "document" embedding.
# It takes in a document (as a string) and a dictionary, en_embeddings
# It processes the document, and looks up the corresponding embedding of each word.
# It then sums them up and returns the sum of all word vectors of that processed tweet.

In [3]:
def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [4]:
def get_document_embedding(tweet, en_embeddings, process_tweet=process_tweet):
    '''
    Input:
        - tweet: a string
        - en_embeddings: a dictionary of word embeddings
    Output:
        - doc_embedding: sum of all word embeddings in the tweet
    '''
    doc_embedding = np.zeros(300)

    # process the document into a list of words (process the tweet)
    processed_doc = process_tweet(tweet)
    for word in processed_doc:
        # add the word embedding to the running total for the document embedding
        if word in en_embeddings.keys():
            word_embedding = en_embeddings.get(word)
        else:
            word_embedding = 0
            
        doc_embedding = doc_embedding + word_embedding

    return doc_embedding

In [5]:
#Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser.
#https://colab.research.google.com

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#  The word embeddings data for English and French words

# The data
# The full dataset for English embeddings is about 3.64 gigabytes, 
# and the French embeddings are about 629 megabytes. 
# To prevent the workspace from crashing, we've extracted a subset of the embeddings for the words that you'll use in this assignment.


#DATA: Add shortcut to your google drive

# en_embeddings.p :
# https://drive.google.com/file/d/1WBZeg8xNSHc27IIKQO5tJ1hNqO88TkAz/view?usp=sharing

# fr_embeddings.p :
# https://drive.google.com/file/d/1nzYKnN2bWCPeL3F_bJEX7KPHSNPqYMua/view?usp=sharing

import pandas as pd
en_embeddings_subset = pd.read_pickle(r'drive/MyDrive/NLP/en_embeddings.p')
fr_embeddings_subset = pd.read_pickle(r'drive/MyDrive/NLP/fr_embeddings.p')

# en_embeddings_subset = pickle.load(open("drive/MyDrive/NLP/en_embeddings.p", "rb"))
# fr_embeddings_subset = pickle.load(open("drive/MyDrive/NLP/fr_embeddings.p", "rb"))

In [7]:
import numpy as np
from nltk.stem import PorterStemmer

import re
import string

import nltk  # Python library for NLP
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer

from nltk.corpus import stopwords


# testing your function
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

tweet_embedding = get_document_embedding(custom_tweet, en_embeddings_subset)
tweet_embedding[-5:]

# Expected output:
# array([-0.00268555, -0.15378189, -0.55761719, -0.07216644, -0.32263184])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


array([-0.00268555, -0.15378189, -0.55761719, -0.07216644, -0.32263184])

In [13]:
# Store all document vectors into a dictionary
# Now, let's store all the tweet embeddings into a dictionary. Implement get_document_vecs()

In [8]:

def get_document_vecs(all_docs, en_embeddings, get_document_embedding=get_document_embedding):
    '''
    Input:
        - all_docs: list of strings - all tweets in our dataset.
        - en_embeddings: dictionary with words as the keys and their embeddings as the values.
    Output:
        - document_vec_matrix: matrix of tweet embeddings.
        - ind2Doc_dict: dictionary with indices of tweets in vecs as keys and their embeddings as the values.
    '''

    # the dictionary's key is an index (integer) that identifies a specific tweet
    # the value is the document embedding for that document
    ind2Doc_dict = {}

    # this is list that will store the document vectors
    document_vec_l = []

    for i, doc in enumerate(all_docs):


        # get the document embedding of the tweet
        doc_embedding = get_document_embedding(doc, en_embeddings)

        # save the document embedding into the ind2Tweet dictionary at index i
        ind2Doc_dict[i] = doc_embedding

        # append the document embedding to the list of document vectors
        document_vec_l.append(ind2Doc_dict[i])



    # convert the list of document vectors into a 2D array (each row is a document vector)
    document_vec_matrix = np.vstack(document_vec_l)

    return document_vec_matrix, ind2Doc_dict

In [9]:
document_vecs, ind2Tweet = get_document_vecs(all_tweets, en_embeddings_subset)

In [10]:
print(f"length of dictionary {len(ind2Tweet)}")
print(f"shape of document_vecs {document_vecs.shape}")

# Expected Output
# length of dictionary 10000
# shape of document_vecs (10000, 300)

length of dictionary 10000
shape of document_vecs (10000, 300)


In [ ]:
#  Looking up the tweets
# Now you will input a tweet, and use cosine similarity to see which tweet in our corpus is similar to your tweet.

In [11]:
my_tweet = 'All good work is done the way ants do things, little by little'
process_tweet(my_tweet)
tweet_embedding = get_document_embedding(my_tweet, en_embeddings_subset)

In [12]:
def cosine_similarity(A, B):
    '''
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        cos: numerical number representing the cosine similarity between A and B.
    '''
    # you have to set this variable to the true label.
    cos = -10
    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)

    return cos

In [13]:

# this gives you a similar tweet as your input.
# this implementation is vectorized...
idx = np.argmax(cosine_similarity(document_vecs, tweet_embedding))
print(idx)
print(all_tweets[idx])

4798
Save your money 
Or use them in a good way
Money is money 
It's for better life
And fun :)
Have a nice day

11:11


In [ ]:
# Finding the most similar tweets with LSH
# I will now implement locality sensitive hashing (LSH) to identify the most similar tweet.

# Instead of looking at all 10,000 vectors, we can just search a subset to find its nearest neighbors.
# we can divide the vector space into regions and search within one region for nearest neighbors of a given vector.# 

In [14]:
N_VECS = len(all_tweets)       # This many vectors.
N_DIMS = len(ind2Tweet[1])     # Vector dimensionality.
print(f"Number of vectors is {N_VECS} and each has {N_DIMS} dimensions.")

Number of vectors is 10000 and each has 300 dimensions.


In [37]:
#  Choosing the number of planes

# * Each plane divides the space to 2 parts.
# * So n planes divide the space into 2^{n} hash buckets.
# * We want to organize 10,000 document vectors into buckets so that every bucket has about ~16 vectors.
# * For that we need {16}/{10000}=625 buckets.
# * We're interested in n, number of planes, so that 2^{n}= 625. Now, we can calculate n=log_{2}625 = 9.29 - approx 10

In [15]:
# The number of planes. We use log2(256) to have ~16 vectors/bucket.
N_PLANES = 10
# Number of times to repeat the hashing to improve the search.
N_UNIVERSES = 25

In [ ]:
# Getting the hash number for a vector
# For each vector, we need to get a unique number associated to that vector in order to assign it to a "hash bucket".

# Hyperlanes in vector spaces
# In  3-dimensional vector space, the hyperplane is a regular plane. In  2  dimensional vector space, the hyperplane is a line.
# Generally, the hyperplane is subspace which has dimension  1  lower than the original vector space has.
# A hyperplane is uniquely defined by its normal vector.
# Normal vector  𝑛  of the plane  𝜋  is the vector to which all vectors in the plane  𝜋  are orthogonal (perpendicular in  3  dimensional case).
# Using Hyperplanes to split the vector space
# We can use a hyperplane to split the vector space into  2  parts.

# All vectors whose dot product with a plane's normal vector is positive are on one side of the plane.
# All vectors whose dot product with the plane's normal vector is negative are on the other side of the plane.

# Encoding hash buckets
# For a vector, we can take its dot product with all the planes, then encode this information to assign the vector to a single hash bucket.
# When the vector is pointing to the opposite side of the hyperplane than normal, encode it by 0.
# Otherwise, if the vector is on the same side as the normal vector, encode it by 1.
# If you calculate the dot product with each plane in the same order for every vector, 
# you've encoded each vector's unique hash ID as a binary number, like [0, 1, 1, ... 0].# 


# We've initialized hash table hashes for you. It is list of N_UNIVERSES matrices, each describes its own hash table. 
#  matrix has N_DIMS rows and N_PLANES columns. Every column of that matrix is a N_DIMS-dimensional normal vector for 
# each of N_PLANES hyperplanes which are used for creating buckets of the particular hash table.

# Exercise: Your task is to complete the function hash_value_of_vector which places vector v in the correct hash bucket.

# First multiply your vector v, with a corresponding plane. This will give you a vector of dimension  (1,N_planes) .
# You will then convert every element in that vector to 0 or 1.
# You create a hash vector by doing the following: if the element is negative, it becomes a 0, otherwise you change it to a 1.
# You then compute the unique number for the vector by iterating over N_PLANES
# Then you multiply  2𝑖  times the corresponding bit (0 or 1).
# You will then store that sum in the variable hash_value.

In [16]:
np.random.seed(0)
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES))
            for _ in range(N_UNIVERSES)]

In [17]:
def hash_value_of_vector(v, planes):
    """Create a hash for a vector; hash_id says which random hash to use.
    Input:
        - v:  vector of tweet. It's dimension is (1, N_DIMS)
        - planes: matrix of dimension (N_DIMS, N_PLANES) - the set of planes that divide up the region
    Output:
        - res: a number which is used as a hash for your vector

    """

    # for the set of planes,
    # calculate the dot product between the vector and the matrix containing the planes
    # remember that planes has shape (300, 10)
    # The dot product will have the shape (1,10)    
    dot_product = np.dot(v, planes)
        
    # get the sign of the dot product (1,10) shaped vector
    sign_of_dot_product = np.sign(dot_product)

    # set h to be false (eqivalent to 0 when used in operations) if the sign is negative,
    # and true (equivalent to 1) if the sign is positive (1,10) shaped vector
    # if the sign is 0, i.e. the vector is in the plane, consider the sign to be positive
    h = sign_of_dot_product >= 0

    # remove extra un-used dimensions (convert this from a 2D to a 1D array)
    h = np.squeeze(h)

    # initialize the hash value to 0
    hash_value = 0

    n_planes = planes.shape[1]
    for i in range(n_planes):
        # increment the hash value by 2^i * h_i        
        hash_value += (2**i) * h[i]
        


    # cast hash_value as an integer
    hash_value = int(hash_value)

    return hash_value

In [18]:
np.random.seed(0)
idx = 0
planes = planes_l[idx]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
print(f" The hash value for this vector,",
      f"and the set of planes at index {idx},",
      f"is {hash_value_of_vector(vec, planes)}")


# Expected Output
# The hash value for this vector, and the set of planes at index 0, is 768

 The hash value for this vector, and the set of planes at index 0, is 768


In [ ]:
# Creating a hash table

# Given that you have a unique number for each vector (or tweet), You now want to create a hash table. 
# You need a hash table, so that given a hash_id, you can quickly look up the corresponding vectors. 
# This allows you to reduce your search by a significant amount of time.

# We have given you the make_hash_table function, which maps the tweet vectors to a bucket and stores the vector there. 
# It returns the hash_table and the id_table. The id_table allows you know which vector in a certain bucket corresponds to what tweet.


In [19]:
def make_hash_table(vecs, planes, hash_value_of_vector=hash_value_of_vector):
    """
    Input:
        - vecs: list of vectors to be hashed.
        - planes: the matrix of planes in a single "universe", with shape (embedding dimensions, number of planes).
    Output:
        - hash_table: dictionary - keys are hashes, values are lists of vectors (hash buckets)
        - id_table: dictionary - keys are hashes, values are list of vectors id's
                            (it's used to know which tweet corresponds to the hashed vector)
    """


    # number of planes is the number of columns in the planes matrix
    num_of_planes = len(planes[0])

    # number of buckets is 2^(number of planes)    
    num_buckets = 2**num_of_planes

    # create the hash table as a dictionary.
    # Keys are integers (0,1,2.. number of buckets)
    # Values are empty lists
    hash_table = {i:[] for i in range(num_buckets)}

    # create the id table as a dictionary.
    # Keys are integers (0,1,2... number of buckets)
    # Values are empty lists
    id_table = {i:[] for i in range(num_buckets)}

    # for each vector in 'vecs'
    for i, v in enumerate(vecs):
        # calculate the hash value for the vector
        h = hash_value_of_vector(v, planes)

        # store the vector into hash_table at key h,
        # by appending the vector v to the list at key h
        hash_table[h].append(v)

        # store the vector's index 'i' (each document is given a unique integer 0,1,2...)
        # the key is the h, and the 'i' is appended to the list at key h
        id_table[h].append(i)



    return hash_table, id_table

In [20]:
planes = planes_l[0]  # get one 'universe' of planes to test the function
tmp_hash_table, tmp_id_table = make_hash_table(document_vecs, planes)

print(f"The hash table at key 0 has {len(tmp_hash_table[0])} document vectors")
print(f"The id table at key 0 has {len(tmp_id_table[0])}")
print(f"The first 5 document indices stored at key 0 of are {tmp_id_table[0][0:5]}")


# Expected output
# The hash table at key 0 has 3 document vectors
# The id table at key 0 has 3
# The first 5 document indices stored at key 0 of are [3276, 3281, 3282]

The hash table at key 0 has 3 document vectors
The id table at key 0 has 3
The first 5 document indices stored at key 0 of are [3276, 3281, 3282]


In [ ]:
# Creating all hash tables
# You can now hash your vectors and store them in a hash table that would allow you to quickly look up and search for similar vectors. 
# Run the cell below to create the hashes. By doing so, you end up having several tables which have all the vectors. 
# Given a vector, you then identify the buckets in all the tables. You can then iterate over the buckets and consider much fewer vectors. 
# The more buckets you use, the more accurate your lookup will be, but also the longer it will take.

In [21]:
# Creating the hashtables
def create_hash_id_tables(n_universes):
    hash_tables = []
    id_tables = []
    for universe_id in range(n_universes):  # there are 25 hashes
        print('working on hash universe #:', universe_id)
        planes = planes_l[universe_id]
        hash_table, id_table = make_hash_table(document_vecs, planes)
        hash_tables.append(hash_table)
        id_tables.append(id_table)
    
    return hash_tables, id_tables

hash_tables, id_tables = create_hash_id_tables(N_UNIVERSES)

working on hash universe #: 0
working on hash universe #: 1
working on hash universe #: 2
working on hash universe #: 3
working on hash universe #: 4
working on hash universe #: 5
working on hash universe #: 6
working on hash universe #: 7
working on hash universe #: 8
working on hash universe #: 9
working on hash universe #: 10
working on hash universe #: 11
working on hash universe #: 12
working on hash universe #: 13
working on hash universe #: 14
working on hash universe #: 15
working on hash universe #: 16
working on hash universe #: 17
working on hash universe #: 18
working on hash universe #: 19
working on hash universe #: 20
working on hash universe #: 21
working on hash universe #: 22
working on hash universe #: 23
working on hash universe #: 24


In [ ]:
# k-Nearest neighbors algorithm
# k-Nearest neighbors algorithm

# k-NN is a method which takes a vector as input and finds the other vectors in the dataset that are closest to it.
# The 'k' is the number of "nearest neighbors" to find (e.g. k=2 finds the closest two neighbors).

# Inputs:

# Vector v,
# A set of possible nearest neighbors candidates
# k nearest neighbors to find.
# The distance metric should be based on cosine similarity.
# cosine_similarity function is already implemented and imported for you. 
# It's arguments are two vectors and it returns the cosine of the angle between them.
# Iterate over rows in candidates, and save the result of similarities between current row and vector v in a python list. 
# Take care that similarities are in the same order as row vectors of candidates.
# Now you can use numpy argsort to sort the indices for the rows of candidates.

In [22]:
def nearest_neighbor(v, candidates, k=1, cosine_similarity=cosine_similarity):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    ### START CODE HERE ###
    similarity_1 = []

    # for each candidate vector...
    for row in candidates:
        # get the cosine similarity
        cos_similarity = cosine_similarity(row, v)

        # append the similarity to the list
        similarity_1.append(cos_similarity)


    
    
    # sort the similarity list and get the indices of the sorted list    
    sorted_ids = np.argsort(similarity_1)
    #print(sorted_ids)
    # Reverse the order of the sorted_ids array
    # sorted_ids = np.flipud(sorted_ids)
    #print(sorted_ids)

    sorted_ids=sorted_ids[-k:]
    sorted_ids=sorted_ids[::-1]
    
    # get the indices of the k most similar candidate vectors
    # k_idx = sorted_ids[:-k+1]
    k_idx = sorted_ids[-k:]
    #k_idx = sorted_ids[:k+1]
    ### END CODE HERE ###
    return k_idx

In [ ]:
# Approximate K-NN

# Implement approximate K nearest neighbors using locality sensitive hashing, 
# to search for documents that are similar to a given document at the index doc_id.

# Inputs
# doc_id is the index into the document list all_tweets.
# v is the document vector for the tweet in all_tweets at index doc_id.
# planes_l is the list of planes (the global variable created earlier).
# k is the number of nearest neighbors to search for.
# num_universes_to_use: to save time, we can use fewer than the total number of available universes. By default, it's set to N_UNIVERSES, which is  25  for this assignment.
# hash_tables: list with hash tables for each universe.
# id_tables: list with id tables for each universe.
# The approximate_knn function finds a subset of candidate vectors that are in the same "hash bucket" as the input vector 'v'. Then it performs the usual k-nearest neighbors search on this subset (instead of searching through all 10,000 tweets).

In [23]:
def approximate_knn(doc_id, v, planes_l, hash_tables, id_tables, k=1, num_universes_to_use=25, hash_value_of_vector=hash_value_of_vector):
    """Search for k-NN using hashes."""
    #assert num_universes_to_use <= N_UNIVERSES

    # Vectors that will be checked as possible nearest neighbor
    vecs_to_consider_l = list()

    # list of document IDs
    ids_to_consider_l = list()

    # create a set for ids to consider, for faster checking if a document ID already exists in the set
    ids_to_consider_set = set()

    # loop through the universes of planes
    for universe_id in range(num_universes_to_use):

        # get the set of planes from the planes_l list, for this particular universe_id
        planes = planes_l[universe_id]

        # get the hash value of the vector for this set of planes
        hash_value = hash_value_of_vector(v, planes)

        # get the hash table for this particular universe_id
        hash_table = hash_tables[universe_id]

        # get the list of document vectors for this hash table, where the key is the hash_value
        document_vectors_l = hash_table[hash_value]

        # get the id_table for this particular universe_id
        id_table = id_tables[universe_id]

        # get the subset of documents to consider as nearest neighbors from this id_table dictionary
        new_ids_to_consider = id_table[hash_value]



        # remove the id of the document that we're searching
        if doc_id in new_ids_to_consider:
            new_ids_to_consider.remove(doc_id)
            print(
                f"removed doc_id {doc_id} of input vector from new_ids_to_search")

        # loop through the subset of document vectors to consider
        for i, new_id in enumerate(new_ids_to_consider):

            # if the document ID is not yet in the set ids_to_consider...
            if new_id not in ids_to_consider_set:
                # access document_vectors_l list at index i to get the embedding
                document_vector_at_i = document_vectors_l[i]
                
                # then append it to the list of vectors to consider as possible nearest neighbors
                vecs_to_consider_l.append(document_vector_at_i)

                # append the new_id (the index for the document) to the list of ids to consider
                ids_to_consider_l.append(new_id)

                # also add the new_id to the set of ids to consider
                # (use this to check if new_id is not already in the IDs to consider)
                ids_to_consider_set.add(new_id)



    # Now run k-NN on the smaller set of vecs-to-consider.
    print("Fast considering %d vecs" % len(vecs_to_consider_l))

    # convert the vecs to consider set to a list, then to a numpy array
    vecs_to_consider_arr = np.array(vecs_to_consider_l)

    # call nearest neighbors on the reduced list of candidate vectors
    nearest_neighbor_idx_l = nearest_neighbor(v, vecs_to_consider_arr, k=k)

    # Use the nearest neighbor index list as indices into the ids to consider
    # create a list of nearest neighbors by the document ids
    nearest_neighbor_ids = [ids_to_consider_l[idx]
                            for idx in nearest_neighbor_idx_l]

    return nearest_neighbor_ids

In [24]:
#document_vecs, ind2Tweet
doc_id = 0
doc_to_search = all_tweets[doc_id]
vec_to_search = document_vecs[doc_id]

In [25]:
# Sample
nearest_neighbor_ids = approximate_knn(doc_id, vec_to_search, planes_l, hash_tables, id_tables, k=3, num_universes_to_use=5)

removed doc_id 0 of input vector from new_ids_to_search
removed doc_id 0 of input vector from new_ids_to_search
removed doc_id 0 of input vector from new_ids_to_search
removed doc_id 0 of input vector from new_ids_to_search
removed doc_id 0 of input vector from new_ids_to_search
Fast considering 77 vecs


In [26]:
print(f"Nearest neighbors for document {doc_id}")
print(f"Document contents: {doc_to_search}")
print("")

for neighbor_id in nearest_neighbor_ids:
    print(f"Nearest neighbor at document id {neighbor_id}")
    print(f"document contents: {all_tweets[neighbor_id]}")

Nearest neighbors for document 0
Document contents: #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

Nearest neighbor at document id 51
document contents: #FollowFriday @France_Espana @reglisse_menthe @CCI_inter for being top engaged members in my community this week :)
Nearest neighbor at document id 701
document contents: With the top cutie of Bohol :) https://t.co/Jh7F6U46UB
Nearest neighbor at document id 2140
document contents: @PopsRamjet come one, every now and then is not so bad :)
